In [1]:
# 하이퍼 파라메터 최적화
# KarasTuner 사용

In [2]:
!pip install keras-tuner -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.0 MB/s eta 0:00:00


In [3]:
# keras tuner 모델 구축
from tensorflow import keras
from tensorflow.keras import layers

In [4]:
def build_model(hp):
  units = hp.Int(name='units', min_value = 16, max_value=64, step = 16)
  model = keras.Sequential([
      layers.Dense(units, activation='relu'),
      layers.Dense(10, activation = 'softmax')
  ])
  optimizer = hp.Choice(name='optimizer', values= ['rmsprop','adam'])
  model.compile(
      optimizer = optimizer,
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy']
  )
  return model

In [5]:
import kerastuner as kt
class SimpleMLP(kt.HyperModel):
  def __init__(self, num_classes):
    self.num_classes = num_classes
  def build(self, hp):
    units = hp.Int(name='units', min_value = 16, max_value=64, step = 16)
    model = keras.Sequential([
        layers.Dense(units, activation='relu'),
        layers.Dense(10, activation = 'softmax')
    ])
    optimizer = hp.Choice(name='optimizer', values= ['rmsprop','adam'])
    model.compile(
        optimizer = optimizer,
        loss = 'sparse_categorical_crossentropy',
        metrics = ['accuracy']
    )
    return model
# 객체 생성
hypermodel = SimpleMLP(num_classes=10)

<ipython-input-5-3c1ecf6c7a9f>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [31]:
tuner = kt.BayesianOptimization(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 2,
    directory = 'mnist_kt_test',
    overwrite = True
)

In [13]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': None}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}


In [14]:
from sklearn.model_selection import train_test_split
(xtr,ytr),(xte,yte) =  keras.datasets.mnist.load_data()

xtr = xtr.reshape(-1,28*28) / 255.
xte = xte.reshape(-1,28*28) / 255.

x_train,x_val,y_train,y_val =  train_test_split(xtr, ytr,random_state=32)
callback = [
    keras.callbacks.EarlyStopping(patience=5)
]

tuner.search(
    x_train,y_train,
    batch_size = 128,
    epochs = 100,
    validation_data = (x_val,y_val),
    callbacks = callback,
    verbose = 2
)


Trial 5 Complete [00h 00m 47s]
val_accuracy: 0.97120001912117

Best val_accuracy So Far: 0.9717333316802979
Total elapsed time: 00h 05m 34s


In [15]:
best_hps =  tuner.get_best_hyperparameters(4)

In [16]:
best_hps

In [30]:
best_hps[3].Choice(name = 'optimizer', values=['rmsprop','adam'])

'rmsprop'

In [27]:
best_hps[3].Int(name='units', min_value = 16, max_value=64, step = 16)

48

In [32]:
# 튜닝후 최상의 조건으로 모델을 생성
model = build_model(best_hps[0])

In [33]:
best_hps[0]._hps

defaultdict(list,
            {'units': [Int(name: "units", min_value: 16, max_value: 64, step: 16, sampling: None, default: 16)],
             'optimizer': [Choice(name: "optimizer", values: ['rmsprop', 'adam'], ordered: False, default: rmsprop)]})

In [34]:
best_hps[0].values

{'units': 64, 'optimizer': 'rmsprop'}

In [35]:
model.get_config()

{'name': 'sequential_2',
 'layers': [{'class_name': 'Dense',
   'config': {'name': 'dense_4',
    'trainable': True,
    'dtype': 'float32',
    'units': 64,
    'activation': 'relu',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}},
  {'class_name': 'Dense',
   'config': {'name': 'dense_5',
    'trainable': True,
    'dtype': 'float32',
    'units': 10,
    'activation': 'softmax',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotUniform',
     'config': {'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'kernel_regularizer': None,
    'bias_regularizer': None,
    'activity_regularizer': None,
    'kernel_constraint': None,
    'bias_constraint': None}}

In [36]:
# 튜닝을 통해서 최상의 신경망 값을 찾았다... dense층에 뉴런의수, 옵티마이져
# 최상의 조건으로 설정된 모델로.. 최상의 epoch수를 찾아

In [37]:
# 최상의 epoch 수를 리턴하는 함수
def get_best_epoch(hp):
  model = build_model(hp)
  callback = [
      keras.callbacks.EarlyStopping(patience=10)
  ]
  history = model.fit(
      x_train,y_train,validation_data = (x_val,y_val), epochs = 100, batch_size=128,callbacks=callback
  )
  val_loss_per_epoch = history.history['val_loss']
  best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
  print(f"best epoch : {best_epoch}")
  return best_epoch

In [ ]:
get_best_epoch(best_hps[0])

Epoch 1/100
352/352 [==============================] - 2s 4ms/step - loss: 0.4360 - accuracy: 0.8822 - val_loss: 0.2863 - val_accuracy: 0.9167
Epoch 2/100
352/352 [==============================] - 1s 4ms/step - loss: 0.2218 - accuracy: 0.9356 - val_loss: 0.2240 - val_accuracy: 0.9350
Epoch 3/100
352/352 [==============================] - 1s 4ms/step - loss: 0.1763 - accuracy: 0.9489 - val_loss: 0.1872 - val_accuracy: 0.9470
Epoch 4/100
352/352 [==============================] - 1s 4ms/step - loss: 0.1459 - accuracy: 0.9578 - val_loss: 0.1621 - val_accuracy: 0.9527
Epoch 5/100
352/352 [==============================] - 2s 5ms/step - loss: 0.1252 - accuracy: 0.9637 - val_loss: 0.1444 - val_accuracy: 0.9573
Epoch 6/100
352/352 [==============================] - 1s 4ms/step - loss: 0.1076 - accuracy: 0.9686 - val_loss: 0.1353 - val_accuracy: 0.9630
Epoch 7/100
324/352 [==========================>...] - ETA: 0s - loss: 0.0959 - accuracy: 0.9722